In [ ]:
import pandas as pd 
df = pd.read_csv('../data/wall_street_journal/WSJ_articles_full.csv')
gdp = pd.read_csv('../data/misc/FRED_US_GDP_1947_2023.csv')

In [ ]:
df = df.sample(n=50000,random_state=42)
df['doc_clean'] = df['doc']
df = df[['doc_clean']]
df = df.dropna().reset_index()

In [ ]:
import sys
sys.path.append('../gtm/')
from corpus import GTMCorpus
from gtm import GTM

# Create a GTMCorpus object
train_dataset = GTMCorpus(
    df,
    vectorizer_args = {
        'stop_words':'english',
        'max_features':50000
    },
    content = '~ 1'
)

In [ ]:
# Train the model
tm = GTM(
    train_dataset, 
    n_topics=50,
    doc_topic_prior='dirichlet', # logistic_normal
    alpha=0.1,
    prevalence_covariates_regularization=0.1,
    update_prior=False,
    encoder_hidden_layers=[], # structure of the encoder neural net
    decoder_hidden_layers=[300], # structure of the decoder neural net
    num_epochs=5,
    print_every=10000,
    log_every=1,
    w_prior=None,
    batch_size=250
)

In [ ]:
# Assess the quality of the learned word embeddings 
# Top 8 closest words to a specific word

import torch
import torch.nn.functional as F

specific_word = 'democrat'

word_id = [i for i,w in enumerate(train_dataset.vocab) if w == specific_word][0]

words = tm.AutoEncoder.decoder['dec_1'].weight.T

logit = torch.matmul(words.T[word_id], words)

beta = F.softmax(logit)

tm.AutoEncoder.eval()
topic_words = []
vals, indices = torch.topk(beta, 8)
vals = vals.cpu().tolist()
indices = indices.cpu().tolist()
[tm.id2token[idx] for idx in indices]